# **First part**

In [ ]:
import pandas as pd
#!pip install requests-html
from requests_html import HTMLSession

In [104]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
session = HTMLSession()

results = session.get(url)

In [132]:
df_list = pd.read_html(results.text) #reading the web pag and assign to a data frame
df = df_list[0] #getting the table

newdf = df[df.Borough != 'Not assigned'].groupby(['Postcode','Borough'],as_index=False).agg(','.join) #creating a new df withoud Not assigned value 

mask = (newdf.Neighbourhood == 'Not assigned') #applying a mask whrn the Neighbourhood is equal to Not assigned it will have the Borough name
newdf.loc[mask,'Neighbourhood'] = newdf.loc[mask,'Borough']

newdf




,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


# **End of the first part**

# **Part 2**

In [134]:
!wget -q -O 'geographical_coordinates.csv' http://cocl.us/Geospatial_data

In [168]:
geo_coo = pd.read_csv('geographical_coordinates.csv')

final_df = pd.merge(newdf, geo_coo, left_on=['Postcode'] ,right_on=['Postal Code'])

del final_df['Postcode']


final_df[['Postal Code', 'Borough', 'Neighbourhood', 'Latitude' , 'Longitude' ]]

final_df.head(20)


,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
5,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
6,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",M1K,43.727929,-79.262029
7,Scarborough,"Clairlea,Golden Mile,Oakridge",M1L,43.711112,-79.284577
8,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",M1M,43.716316,-79.239476
9,Scarborough,"Birch Cliff,Cliffside West",M1N,43.692657,-79.264848


# **End of part 2**

# **Part 3**

In [143]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# **Get the Toronto Borough and assingning to a new DataFrame**

In [170]:
for i in final_df['Borough']:
     j = i.split(" ")
    
     try:
         if j[1] == 'Toronto':
            toronto_data = final_df[final_df['Borough'] == i].reset_index(drop=True)
     except:
        None
toronto_data.head()

,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,East Toronto,The Beaches,M4E,43.676357,-79.293031
1,East Toronto,"The Danforth West,Riverdale",M4K,43.679557,-79.352188
2,East Toronto,"The Beaches West,India Bazaar",M4L,43.668999,-79.315572
3,East Toronto,Studio District,M4M,43.659526,-79.340923
4,East Toronto,Business Reply Mail Processing Centre 969 Eastern,M7Y,43.662744,-79.321558


In [144]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [145]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(final_df['Latitude'], final_df['Longitude'], final_df['Borough'], final_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [146]:
CLIENT_ID = '30IWN5SH5ZAAH4PSP4QOVCPKR5SSW1KZMRNK2DM4UDHBWOKN' # your Foursquare ID
CLIENT_SECRET = 'NKMDDROD5PFUVPFR44QQAI5PX2KA5ZZ5TDQR5YIRZYJLIFX1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 30IWN5SH5ZAAH4PSP4QOVCPKR5SSW1KZMRNK2DM4UDHBWOKN
CLIENT_SECRET:NKMDDROD5PFUVPFR44QQAI5PX2KA5ZZ5TDQR5YIRZYJLIFX1


In [172]:
toronto_data.loc[0, 'Borough']
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [173]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius
 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=30IWN5SH5ZAAH4PSP4QOVCPKR5SSW1KZMRNK2DM4UDHBWOKN&client_secret=NKMDDROD5PFUVPFR44QQAI5PX2KA5ZZ5TDQR5YIRZYJLIFX1&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [174]:
results = requests.get(url).json()

In [175]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [177]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


In [179]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [189]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Business Reply Mail Processing Centre 969 Eastern


In [183]:
print(toronto_venues.shape)
toronto_venues.head()

(122, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Domino's Pizza,43.679058,-79.297382,Pizza Place
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [184]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
Studio District,39,39,39,39,39,39
The Beaches,5,5,5,5,5,5
"The Beaches West,India Bazaar",20,20,20,20,20,20
"The Danforth West,Riverdale",42,42,42,42,42,42


In [185]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 68 uniques categories.


In [190]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,American Restaurant,Auto Workshop,Bakery,Bank,Bar,Board Shop,Bookstore,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Juice Bar,Latin American Restaurant,Light Rail Station,Liquor Store,Middle Eastern Restaurant,Movie Theater,Neighborhood,Park,Pet Store,Pizza Place,Pub,Recording Studio,Restaurant,Sandwich Place,Seafood Restaurant,Skate Park,Spa,Sports Bar,Stationery Store,Steakhouse,Sushi Restaurant,Thai Restaurant,Thrift / Vintage Store,Trail
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [191]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,American Restaurant,Auto Workshop,Bakery,Bank,Bar,Board Shop,Bookstore,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Juice Bar,Latin American Restaurant,Light Rail Station,Liquor Store,Middle Eastern Restaurant,Movie Theater,Park,Pet Store,Pizza Place,Pub,Recording Studio,Restaurant,Sandwich Place,Seafood Restaurant,Skate Park,Spa,Sports Bar,Stationery Store,Steakhouse,Sushi Restaurant,Thai Restaurant,Thrift / Vintage Store,Trail
0,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.0625,0.000000,0.000000,0.000000,0.00,0.000000,0.062500,0.00000,0.00,0.0625,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,0.000000,0.00000,0.000000,0.00000,0.000000,0.0625,0.0625,0.00,0.000000,0.00,0.00000,0.000000,0.0625,0.0625,0.000000,0.000000,0.000000,0.00000,0.00,0.062500,0.0,0.000000,0.00000,0.000000,0.00000,0.000000,0.125,0.00000,0.000000,0.00,0.062500,0.00,0.06250,0.00000,0.0625,0.06250,0.000000,0.000000,0.0625,0.00000,0.00000,0.000000,0.00,0.00,0.000000,0.000000,0.00000
1,Studio District,0.025641,0.051282,0.0000,0.051282,0.025641,0.025641,0.00,0.025641,0.025641,0.00000,0.00,0.0000,0.102564,0.00000,0.025641,0.025641,0.025641,0.076923,0.025641,0.0000,0.025641,0.00000,0.025641,0.00000,0.025641,0.0000,0.0000,0.00,0.025641,0.00,0.00000,0.000000,0.0000,0.0000,0.025641,0.025641,0.000000,0.00000,0.00,0.025641,0.0,0.025641,0.00000,0.051282,0.00000,0.025641,0.000,0.00000,0.025641,0.00,0.025641,0.00,0.00000,0.00000,0.0000,0.00000,0.025641,0.025641,0.0000,0.00000,0.00000,0.025641,0.00,0.00,0.025641,0.025641,0.00000
2,The Beaches,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.00,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,0.000000,0.00000,0.000000,0.0000,0.0000,0.00,0.000000,0.00,0.00000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.2,0.000000,0.00000,0.000000,0.00000,0.000000,0.000,0.00000,0.000000,0.00,0.000000,0.00,0.20000,0.20000,0.0000,0.00000,0.000000,0.000000,0.0000,0.00000,0.00000,0.000000,0.00,0.00,0.000000,0.000000,0.20000
3,"The Beaches West,India Bazaar",0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.05,0.000000,0.050000,0.00000,0.05,0.0500,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,0.000000,0.00000,0.000000,0.0000,0.0500,0.05,0.000000,0.05,0.00000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.00000,0.05,0.000000,0.0,0.050000,0.00000,0.050000,0.00000,0.000000,0.000,0.05000,0.000000,0.05,0.050000,0.05,0.05000,0.05000,0.0000,0.00000,0.100000,0.000000,0.0000,0.00000,0.00000,0.000000,0.05,0.05,0.000000,0.000000,0.00000
4,"The Danforth West,Riverdale",0.023810,0.023810,0.0000,0.023810,0.000000,0.000000,0.00,0.023810,0.023810,0.02381,0.00,0.0000,0.023810,0.02381,0.000000,0.000000,0.000000,0.095238,0.000000,0.0000,0.000000,0.02381,0.000000,0.02381,0.023810,0.0000,0.0000,0.00,0.000000,0.00,0.02381,0.047619,0.0000,0.0000,0.000000,0.000000,0.190476,0.02381,0.00,0.000000,0.0,0.071429,0.02381,0.071429,0.02381,0.000000,0.000,0.02381,0.000000,0.00,0.000000,0.00,0.02381,0.02381,0.0000,0.02381,0.000000,0.000000,0.0000,0.02381,0.02381,0.000000,0.00,0.00,0.000000,0.000000,0.02381


In [192]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0    Light Rail Station  0.12
1         Garden Center  0.06
2            Skate Park  0.06
3  Gym / Fitness Center  0.06
4                Garden  0.06


----Studio District----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2               Bakery  0.05
3   Italian Restaurant  0.05
4  American Restaurant  0.05


----The Beaches----
                venue  freq
0               Trail   0.2
1         Pizza Place   0.2
2                 Pub   0.2
3   Health Food Store   0.2
4  Italian Restaurant   0.0


----The Beaches West,India Bazaar----
                venue  freq
0      Sandwich Place  0.10
1                 Pub  0.05
2      Ice Cream Shop  0.05
3  Italian Restaurant  0.05
4   Food & Drink Shop  0.05


----The Danforth West,Riverdale----
                    venue  freq
0        Greek Restaurant  0.19
1             Coffee Shop  0.10
2          Ice Cream Shop  0.0

In [193]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [194]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Garden Center,Brewery,Comic Shop,Gym / Fitness Center,Burrito Place,Park,Pizza Place,Recording Studio,Restaurant
1,Studio District,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Yoga Studio,Cheese Shop,Chinese Restaurant,Ice Cream Shop,Clothing Store
2,The Beaches,Trail,Pub,Pizza Place,Health Food Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Diner,Dessert Shop,Clothing Store
3,"The Beaches West,India Bazaar",Sandwich Place,Burger Joint,Pizza Place,Fish & Chips Shop,Food & Drink Shop,Gym,Ice Cream Shop,Italian Restaurant,Burrito Place,Liquor Store
4,"The Danforth West,Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Dessert Shop,Indian Restaurant,Grocery Store,Fruit & Vegetable Store,Diner


In [202]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 4, 1, 2, 0], dtype=int32)

In [204]:
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Borough,Neighbourhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Beaches,M4E,43.676357,-79.293031,1,Trail,Pub,Pizza Place,Health Food Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Diner,Dessert Shop,Clothing Store
1,East Toronto,"The Danforth West,Riverdale",M4K,43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Dessert Shop,Indian Restaurant,Grocery Store,Fruit & Vegetable Store,Diner
2,East Toronto,"The Beaches West,India Bazaar",M4L,43.668999,-79.315572,2,Sandwich Place,Burger Joint,Pizza Place,Fish & Chips Shop,Food & Drink Shop,Gym,Ice Cream Shop,Italian Restaurant,Burrito Place,Liquor Store
3,East Toronto,Studio District,M4M,43.659526,-79.340923,4,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Yoga Studio,Cheese Shop,Chinese Restaurant,Ice Cream Shop,Clothing Store
4,East Toronto,Business Reply Mail Processing Centre 969 Eastern,M7Y,43.662744,-79.321558,3,Light Rail Station,Garden Center,Brewery,Comic Shop,Gym / Fitness Center,Burrito Place,Park,Pizza Place,Recording Studio,Restaurant


In [206]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## **Examine the cluster**

### Cluster 1

In [207]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"The Danforth West,Riverdale",0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Dessert Shop,Indian Restaurant,Grocery Store,Fruit & Vegetable Store,Diner


### Cluster 2

In [209]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,1,Trail,Pub,Pizza Place,Health Food Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Diner,Dessert Shop,Clothing Store


### Cluster 3

In [210]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"The Beaches West,India Bazaar",2,Sandwich Place,Burger Joint,Pizza Place,Fish & Chips Shop,Food & Drink Shop,Gym,Ice Cream Shop,Italian Restaurant,Burrito Place,Liquor Store


# **END of part 3**